Why to represent Texts into Vectors?

Ans : Any kind of machine learning, deep learning or statistical learning platform understands only numbers.

How to represent Texts into Vectors?

1. One Hot Encoding
2. Bag of Words Model
3. TF-IDF
4. Word2Vec
5. FastText
6. GLOve
7. BERT

References

https://www.analyticsvidhya.com/blog/2023/07/step-by-step-guide-to-word2vec-with-gensim/

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

https://radimrehurek.com/gensim/models/word2vec.html

https://analyticsindiamag.com/word2vec-vs-glove-a-comparative-guide-to-word-embedding-techniques/

https://medium.com/intelligentmachines/word-embedding-and-one-hot-encoding-ad17b4bbe111



# Install Libraries

In [1]:
!pip install --upgrade gensim -qqq
!pip install scikit-learn -qqq
!pip install swifter -qqq
!pip install glove-python3 -qqq
!pip install spacy -qqq

# Prepare Data

In [1]:
import pandas as pd

train = pd.read_csv("train.csv", on_bad_lines = 'skip')
test = pd.read_csv("test.csv", on_bad_lines = 'skip')

In [2]:
import spacy
import string

nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [3]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):

    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    return mytokens


In [4]:
#from sklearn.utils import shuffle
#train = shuffle(train)

print(train.shape)
train = train.sample(frac = 0.01)
print(train.shape)

print(test.shape)
test = test.sample(frac = 0.01)
print(test.shape)

(159571, 8)
(1596, 8)
(153164, 9)
(1532, 9)


In [5]:
train['tokens'] = train['comment_text'].apply(spacy_tokenizer)
test['tokens'] = test['comment_text'].apply(spacy_tokenizer)

In [6]:
train['sent'] = train['tokens'].str.join(' ')
test['sent'] = test['tokens'].str.join(' ')

# Vectorize Data

1. Bag Of Words Model

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vectors= vectorizer.fit_transform(train['sent'])
X_test_vectors= vectorizer.transform(test['sent'])

print(X_train_vectors.shape)
print(X_test_vectors.shape)

(1596, 9831)
(1532, 9831)


In [8]:
len(vectorizer.get_feature_names_out())

9831

3. TF-IDF Model

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(train['sent'])
X_test_vectors = vectorizer.transform(test['sent'])

print(X_train_vectors.shape)
print(X_test_vectors.shape)

(1596, 9831)
(1532, 9831)


In [10]:
len(vectorizer.get_feature_names_out())

9831

4. Word2Vec

In [11]:
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [12]:
def wv_sent_vec(sent, model):

    vector_size = model.wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 0
    for w in sent:
        if w in model.wv: #model.wv.index_to_key
            ctr += 1
            wv_res += model.wv[w]
    if ctr != 0:
      wv_res = wv_res/ctr
    else:
      wv_res = wv_res
    return wv_res

In [ ]:
# w2v_model = Word2Vec(min_count=1,
#                      window=2,
#                      vector_size=300,
#                      sample=6e-5,
#                      alpha=0.03,
#                      min_alpha=0.0007,
#                      negative=20,
#                      workers=4)

In [ ]:
# w2v_model.build_vocab(df['tokens']) #, progress_per=10000

In [ ]:
# w2v_model.train(df['tokens'], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [15]:
import gensim.models

model = gensim.models.Word2Vec(sentences = train['tokens'], min_count = 5, compute_loss = True)
print(model.get_latest_training_loss())

model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")

204930.8125


In [16]:
train['wv_vec'] = train['tokens'].apply(wv_sent_vec, args = (model,))
test['wv_vec'] = test['tokens'].apply(wv_sent_vec, args = (model,))

In [18]:
len(train['wv_vec'].values[0])

100

5. FastText

In [19]:
from gensim.models import FastText


embedding_size = 300
window_size = 5
#min_word = 5
#down_sampling = 1e-2
model = FastText(vector_size=embedding_size, window=window_size, min_count=1, sentences=train['tokens'], epochs=1)

In [ ]:
# from gensim.test.utils import get_tmpfile
# model.save(get_tmpfile("fasttext.model"))
# model = FastText.load(get_tmpfile("fasttext.model"))

In [20]:
model.save("fasttext.model")
model = FastText.load("fasttext.model")

In [21]:
train['ft_vec'] = train['tokens'].apply(wv_sent_vec, args = (model,))
test['ft_vec'] = test['tokens'].apply(wv_sent_vec, args = (model,))

In [22]:
len(train['ft_vec'].values[0])

300

6. GLOve

In [24]:
from glove import Corpus, Glove

#Creating a corpus object
corpus = Corpus()

#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus.fit(list(train['tokens']), window=10)

glove = Glove(no_components=300, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=5, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 5 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [25]:
def gl_sent_vec(sent):
    vector_size = glove.no_components
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 0
    for w in sent:
        if w in glove.dictionary.keys():
            ctr += 1
            wv_res += glove.word_vectors[glove.dictionary[w]]
    if ctr != 0:
      wv_res = wv_res/ctr
    return wv_res

In [26]:
model = glove.load('glove.model')

In [28]:
#Test for random word in corpus

len(glove.word_vectors[glove.dictionary['document']])

300

In [29]:
train['glovevec'] = train['tokens'].apply(gl_sent_vec)
test['glovevec'] = test['tokens'].apply(gl_sent_vec)